# MRETT Series 2 - Leam Lou Racing Squad Results

JSON query to get results from RGTDB: https://rgtdb.com/results/json?search=TLR%20MRETT&offset=0&limit=10 

The search string is "TLR MRETT". I'm glad we used consistent naming conventions. 

## Get data, cache results

In [135]:
import pandas as pd 

import datetime
from datetime import datetime as dt

import io
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("Debug level logging turned on")

import requests
from cachecontrol import CacheControl
from cachecontrol.caches import FileCache
from cachecontrol.heuristics import ExpiresAfter

sess = requests.session()
cached_sess = CacheControl(sess, cache = FileCache('.web_cache'), heuristic=ExpiresAfter(hours=1))

try:
    response = cached_sess.get('https://rgtdb.com/results/json?search=TLR%20MRETT&offset=0&limit=10') # Search string = TLR MRETT
    response.raise_for_status()

except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

logger.setLevel(logging.ERROR)

print(dt.now())

DEBUG:root:Debug level logging turned on
DEBUG:cachecontrol.controller:Looking up "https://rgtdb.com/results/json?search=TLR%20MRETT&offset=0&limit=10" in the cache
DEBUG:cachecontrol.controller:Current age based on date: 32162
DEBUG:cachecontrol.controller:Freshness lifetime from expires: 3601
DEBUG:cachecontrol.controller:The cached response is "stale" with no etag, purging
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): rgtdb.com:443
DEBUG:urllib3.connectionpool:https://rgtdb.com:443 "GET /results/json?search=TLR%20MRETT&offset=0&limit=10 HTTP/1.1" 200 None
DEBUG:cachecontrol.controller:Updating cache with response from "https://rgtdb.com/results/json?search=TLR%20MRETT&offset=0&limit=10"
DEBUG:cachecontrol.controller:Caching b/c of expires header


2021-05-24 18:36:59.379383


In [136]:
rides = response.json()['rows']

for row in rides:
    print(row['detailsUrl'])
    if row['detailsUrl'] == '/events/87427':
        row['detailsUrl'] = '/events/88524' # Substitute event ID: 88524
        print('Substituted:', row['detailsUrl'])
        
    row['dfs'] = pd.read_html('https://rgtdb.com' + row['detailsUrl'])

/events/92699
/events/91479
/events/90165
/events/88829
/events/87427
Substituted: /events/88524
/events/86163


## Pseudo-Anonymize Rider Names

Use key generated with Python secrets lib, stored in separate module `pseudo_key` in variable `pseudoKey`.

* https://docs.python.org/3/library/secrets.html

Pseudo-anonymous hash generated with `hashlib` digest `blake2b` alogrithm.

* https://docs.python.org/3/library/hashlib.html

In [137]:
import pseudo_key
import hashlib

def pseudo_anon_name(name):
    hashGen = hashlib.blake2b(key=pseudo_key.pseudoKey, digest_size=10) # sha256
    hashGen.update(name.encode('UTF-8'))
    hash = hashGen.hexdigest()
    return hash

pseudo_anon_name('test')

'b7c70dcf79babeef0c62'

In [139]:
for row in rides:
    for dframe in row['dfs']:
        dframe['Name'] = dframe['Name'].apply(lambda x: pseudo_anon_name(x))

## Convert Name Hashes to "Cool Names"

https://pypi.org/project/coolname/

In [140]:
#!pip install coolname --upgrade

In [142]:
from coolname import generate_slug

df_results = pd.DataFrame()
unique_names = { }

for row in rides:
    df_row = row['dfs'][1]
    df_row['Ride'] = row['name']
    df_results = df_results.append(df_row.loc[df_row['#'] == 3])
    for dframe in row['dfs']:
        for name in dframe['Name'].values:
            if not(name in unique_names.keys()):
                unique_names[name] = generate_slug(2)            

df_names = pd.DataFrame(unique_names.items(), columns=['Name', 'Pseudonym'])
df_results = df_results.merge(df_names, how='left')
df_results = df_results[::-1].reset_index()


## Create Plots

In [189]:
import pandas_bokeh
import bokeh
from bokeh.themes import built_in_themes
from bokeh.io import curdoc

pandas_bokeh.output_notebook()

Loading BokehJS ...

In [181]:
df_results.plot_bokeh(y='Avg wKg', figsize=(1500, 350),
                                            title="3rd Place Rider W/kg per Ride",
                                            xlabel="Team Time Trial",
                                            plot_data_points=True,
                                            hovertool_string="""<h2> @{Ride}</h2>
                                                            Rider: <b>@{Pseudonym}</b><br>
                                                            Time: <b>@{Finish Time}</b>"""
)

Figure(id='7877', ...)

In [145]:
df_results[['Ride', 'Pseudonym',  'Finish Time', 'Avg wKg']]

,Ride,Pseudonym,Finish Time,Avg wKg
0,TLR MRETT21 Monday Race,tactful-clam,37:16.5 + 0.986,3.38
1,TLR MRETT22 Race Monday,tactful-clam,34:59.8 + 1.063,3.50
2,TLR MRETT23 Monday Race,loutish-antelope,32:04.1 + 1.138,4.08
3,TLR MRETT24 Monday Race,loutish-antelope,35:31.8 + 0.604,3.72
4,TLR MRETT25 Race Monday,tactful-clam,36:55.0 + 2.386,3.50
5,TLR MRETT26 Race Monday,loutish-antelope,37:20.9 + 0.127,3.78


In [180]:
df_races = pd.DataFrame()

for row in reversed(rides):
    df_row = row['dfs'][1]
    df_row['Ride'] = row['name']
    df_row = df_row.merge(df_names, how='left')
    print(row['name'])
    print(df_row[['#', 'Pseudonym', 'Finish Time', 'Avg wKg']])
    print('--------------------------------------------------\n')
    df_races = df_races.append(df_row)


TLR MRETT21 Monday Race
   #          Pseudonym         Finish Time  Avg wKg
0  1      eager-meerkat             37:15.5     4.34
1  2  persimmon-jaybird     37:16.0 + 0.471     3.08
2  3       tactful-clam     37:16.5 + 0.986     3.38
3  4     watchful-panda    37:31.0 + 15.448     3.58
4  5      amaranth-myna  46:27.4 + 9:11.907     2.45
5  6  belligerent-swift  46:35.8 + 9:20.252     1.96
--------------------------------------------------

TLR MRETT22 Race Monday
   #            Pseudonym         Finish Time  Avg wKg
0  1  idealistic-porpoise             34:58.7     3.81
1  2        eager-meerkat     34:59.7 + 0.974     4.46
2  3         tactful-clam     34:59.8 + 1.063     3.50
3  4     loutish-antelope     34:59.8 + 1.093     3.96
4  5       watchful-panda  38:10.4 + 3:11.689     3.48
5  6    persimmon-jaybird  39:01.3 + 4:02.522     3.23
--------------------------------------------------

TLR MRETT23 Monday Race
   #            Pseudonym         Finish Time  Avg wKg
0  1  idealis

In [168]:
df_wkg = df_races.reset_index().pivot(index='Ride', columns='Pseudonym')['Avg wKg']
df_wkg

Pseudonym,amaranth-myna,belligerent-swift,eager-meerkat,idealistic-porpoise,loutish-antelope,persimmon-jaybird,tactful-clam,watchful-panda
Ride,,,,,,,,
TLR MRETT21 Monday Race,2.45,1.96,4.34,NaN,NaN,3.08,3.38,3.58
TLR MRETT22 Race Monday,NaN,NaN,4.46,3.81,3.96,3.23,3.50,3.48
TLR MRETT23 Monday Race,NaN,NaN,4.39,3.54,4.08,3.14,3.22,3.54
TLR MRETT24 Monday Race,NaN,NaN,4.33,3.71,3.72,3.17,3.26,3.32
TLR MRETT25 Race Monday,NaN,NaN,4.31,3.57,NaN,3.20,3.50,3.45
TLR MRETT26 Race Monday,NaN,NaN,NaN,NaN,3.78,3.12,3.35,3.26


In [190]:
df_wkg.fillna(0).plot_bokeh(figsize=(1500, 700), plot_data_points=True, ylabel='W/kg', title="All Riders W/kg", legend='bottom_center')

Figure(id='15204', ...)